In [106]:
import pandas as pd
import numpy as np
from collections import defaultdict
pd.options.display.max_columns = 999

grade_scale = {'A+': 13, 'A': 12, 'A-': 11, 'B+': 10, 'B': 9, 'B-': 8, 'C+': 7, 'C': 6, 'C-': 5, 'D+': 4, 'D': 3, 'D-': 2, 'F': 1, 'NA': 0}

courses = {"0":1, "1":2, "3":2, "3+": 4}
courses_imp = {'Most important': 4, 'Second-most important': 3, 'Third-most important': 2, 'Fourth-most important': 1}
courses_int = {'Most interesting': 4, 'Second-most interesting': 3, 'Third-most interesting': 2, 'Fourth-most interesting': 1}

scale_map = {"courses": courses, "courses_imp": courses_imp, "courses_int": courses_int}

In [127]:
df = pd.read_csv("data/class_data.csv")
df_interested = df[df["pre_studyinterest"] == "Yes"]
df_var_sel = pd.read_excel("data/student_interested_var_decision.xlsx", sheet_name = None)
df_codebook = pd.read_excel("data/codebook.xlsx", sheet_name = None)

In [37]:
df_interested.head()

,randomid,intervention,asia,roster6_21,roster6_29,roster7_2,courseformat,coursecode,departmentnamecode,coursenum,...,wcoursereg4,woactreg31,woactreg32,wcoursesat4,woactsat31,woactsat32,wdistractw5,wwhenw5,wstrategyw5,statusw5
0,1,1.0,1.0,1.0,1.0,1.0,1.0,5005.0,BIO SCI,9B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
4,5,1.0,0.0,1.0,1.0,1.0,1.0,5005.0,BIO SCI,9B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no
6,7,2.0,1.0,1.0,1.0,1.0,1.0,5005.0,BIO SCI,9B,...,6.8,6.6,4.8,7.0,7.0,6.5,Not having enough time to understand all the c...,I think every day can be an obstacle because I...,I try to have a healthy schedule and wake up e...,yes
8,9,2.0,0.0,1.0,1.0,1.0,1.0,5005.0,BIO SCI,9B,...,7.0,7.0,5.0,7.0,7.0,7.0,Sleeping poorly,At night,"Iâm underslept, so unsure how to respond",yes
9,10,1.0,0.0,1.0,1.0,1.0,1.0,5005.0,BIO SCI,9B,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes


In [116]:
pre_post_vars = df_var_sel["Pre & Post Keep"]["Var"].tolist()
pre_post_dict = defaultdict(list)
for c, name_stem in zip(df_var_sel["Pre & Post Keep"]["Var"], df_var_sel["Pre & Post Keep"]["Name Stem"]):
    pre_post_dict[name_stem].append(c.replace(name_stem, '').replace("_", ''))
df_pre_post_pair = df_interested[["randomid"] + pre_post_vars].copy().reset_index(drop = True)
df_pre_post_pair["pre_post_count"] = 0
for name_stem in pre_post_dict:
    if len(pre_post_dict[name_stem]) == 2:
        pre_index = df_pre_post_pair[df_pre_post_pair["pre_{}".format(name_stem)].notnull()].index
        post_index = df_pre_post_pair[df_pre_post_pair["post_{}".format(name_stem)].notnull()].index
        both_index = set(pre_index).intersection(set(post_index))
        df_pre_post_pair.loc[both_index, "pre_post_count"] += 1
df_pre_post_pair["pre_post_count"] /= temp_df.shape[0]

for name_stem in pre_post_dict:
    if len(pre_post_dict[name_stem]) == 2:
        column_map = grade_scale if "grade" in name_stem else scale_map[name_stem] if name_stem in scale_map else None
        if "grade" in name_stem or name_stem in scale_map:
            df_pre_post_pair["pre_{}".format(name_stem)] = df_pre_post_pair["pre_{}".format(name_stem)].map(column_map)
            df_pre_post_pair["post_{}".format(name_stem)] = df_pre_post_pair["post_{}".format(name_stem)].map(column_map)

        
df_pre_post_pair = df_pre_post_pair[df_pre_post_pair["pre_post_count"] >= 0.5]
df_pre_post_pair_describe = df_pre_post_pair.describe()

In [135]:
pre_post_inc, pre_post_dec = {}, {}
for name_stem in pre_post_dict:
    if len(pre_post_dict[name_stem]) == 2:
        if "pre_{}".format(name_stem) not in df_pre_post_pair_describe.columns or \
            "post_{}".format(name_stem) not in df_pre_post_pair_describe.columns:
            print(name_stem)
        else:
            pre = df_pre_post_pair_describe.loc["mean", "pre_{}".format(name_stem)]
            post = df_pre_post_pair_describe.loc["mean", "post_{}".format(name_stem)]
            if pre > post:
                pre_post_dec[name_stem] = post - pre
            else:
                pre_post_inc[name_stem] = post - pre

start
end
distribution


In [136]:
temp = pd.DataFrame({"Name Stem": list(pre_post_dec.keys()), "Difference": list(pre_post_dec.values())})
temp.merge(df_codebook["Pre & Post"][["Name Stem", "Item", "Label", "Response Values", "Construct"]],
                                        left_on = "Name Stem", right_on = "Name Stem")

,Name Stem,Difference,Item,Label,Response Values,Construct
0,agqmap1,-0.258209,I want to learn as much as possible in this cl...,want to learn as much as possible,"slider: 1=Not at all true of me, 7 = Very true...",Achievement Goals (Mastery approach)
1,agqmap2,-0.185075,It is important for me to understand the conte...,want to understand the content thoroughly,"slider: 1=Not at all true of me, 7 = Very true...",Achievement Goals (Mastery approach)
2,agqmap5,-0.065061,Sometimes I’m afraid that I may not understand...,afraid may not understand everything,"slider: 1=Not at all true of me, 7 = Very true...",Achievement Goals (Mastery avoid)
3,agqmap6,-0.888889,It is important for me to do well compared to ...,important to do well compared to others,"slider: 1=Not at all true of me, 7 = Very true...",Achievement Goals (Performance approach)
4,agqmap3,-1.044792,It is important for me not to do worse than th...,important not to do worse than others,"slider: 1=Not at all true of me, 7 = Very true...",Achievement Goals (Performance avoid)
5,agqmap16,-0.427183,I don’t want to look like I’m not as smart as ...,don't want to look less smart,"slider: 1=Not at all true of me, 7 = Very true...",Achievement Goals (Performance avoid)
6,att1,-0.024242,"How important to you, personally, is it to be ...",important to be a cooking science person,"slider: 1=Not at all important, 7 = Very impor...",Attainment Value
7,cost8,-0.168657,How much time will you have for this class aft...,time left for class after other activities,"slider: 1=Not nearly enough, 7 = Enough",Cost Value (Outside Effort)
8,cost9,-0.305970,How much effort will you have left for this cl...,effort left for class after other activities,"slider: 1=Not nearly enough, 7 = Enough",Cost Value (Outside Effort)
9,studydays,-0.313433,How many days of each week will you work on th...,days per week you will work on course,1 to 7,Course Plan


In [134]:
temp = pd.DataFrame({"Name Stem": list(pre_post_inc.keys()), "Difference": list(pre_post_inc.values())})
temp.merge(df_codebook["Pre & Post"][["Name Stem", "Item", "Label", "Response Values", "Construct"]],
                                        left_on = "Name Stem", right_on = "Name Stem")

,Name Stem,Difference,Item,Label,Response Values,Construct
0,agqmap10,0.048112,I am often concerned that I may not learn all ...,concerned haven't learned everything,"slider: 1=Not at all true of me, 7 = Very true...",Achievement Goals (Mastery avoid)
1,agqmap17,0.018779,It’s important to me that others think I’m sma...,important I look smart to others,"slider: 1=Not at all true of me, 7 = Very true...",Achievement Goals (Performance approach)
2,agqmap7,0.026911,I just want to avoid doing poorly in this class,just want to avoid doing poorly,"slider: 1=Not at all true of me, 7 = Very true...",Achievement Goals (Performance avoid)
3,att2,0.188558,How important is it that others see you as kno...,important to be seen as cooking science person,"slider: 1=Not at all important, 7 = Very impor...",Attainment Value
4,att3,0.308160,How important to your identity is it to be kno...,important to my identity to know cooking science,"slider: 1=Not at all important, 7 = Very impor...",Attainment Value
5,cost1,0.323697,How stressful will this class be?,class will be stressful,"slider: 1=Not at all stressful, 7 = Very stres...",Cost Value (Emotional)
6,cost2,0.690769,How frustrating will this class be?,class will be frustrating,"slider: 1=Not at all frustrating, 7 = Very fur...",Cost Value (Emotional)
7,cost3,0.618973,How emotionally draining will this class be?,class will be emotionally draining,"slider: 1=Not at all draining, 7 = Very draining",Cost Value (Emotional)
8,cost4,0.203279,How much do you have to sacrifice to do well i...,have to sacrifice a lot to do well,"slider: 1=Nothing, 7 = An incredible amount",Cost Value (Loss of Valued Alternatives)
9,cost5,0.439062,How many other valued activities does this cla...,have to give up valued activities,"slider: 1=None, 7 = An incredible amount",Cost Value (Loss of Valued Alternatives)


In [150]:
pre_post_inc.update(pre_post_dec)
temp = pd.DataFrame({"Name Stem": list(pre_post_inc.keys()), "Post - Pre": list(pre_post_inc.values())})
temp.merge(df_codebook["Pre & Post"][["Name Stem", "Item", "Label", "Response Values", "Construct"]],
                                        left_on = "Name Stem", right_on = "Name Stem").sort_values("Construct")\
                                                                                      .reset_index(drop = True)\
                                                                                      .to_csv("data/Pre_Post_Difference.csv", index = False)